In [3]:
# !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
# !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
# !pip install --no-deps unsloth

In [4]:
# !pip install -U langchain langchain-community
# !pip install faiss-cpu
# !pip install rank_bm25
# !pip install bert_score

In [2]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
finetune_model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "foxxar04/llama_qlora1",
    max_seq_length = 8096,
    dtype = None,
    load_in_4bit = True,
    device_map="auto",
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.5.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [2]:
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from sentence_transformers import CrossEncoder
from langchain_core.runnables import chain, RunnableConfig, RunnableParallel, RunnablePassthrough, Runnable, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from typing import List, Dict
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

In [5]:
pipe = pipeline(
  "text-generation",
  model=finetune_model,
  tokenizer=tokenizer,
  max_new_tokens=8096,  # Максимальная длина генерируемого текста
  temperature=0.3,     # Температура для контроля случайности
  repetition_penalty=1.15, # Предотвращение повторений
  top_p = 0.90,
)

llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

In [6]:
class LegalAgent:
  def __init__(self, llm, db_path, embedder_name, reranker_name, top_k):

    self.top_k = top_k
    self.weights = [0.4, 0.6]

    self.embedder = HuggingFaceEmbeddings(
      model_name=embedder_name
    )
    self.vector_db = FAISS.load_local(
        db_path, self.embedder, allow_dangerous_deserialization=True
    )
    self.db_retriever = self.vector_db.as_retriever(
        search_kwargs={
            "k":self.top_k
            }
    )
    self.bm25_retriever = BM25Retriever.from_documents(
        list(self.vector_db.docstore._dict.values()),
        metadata_keys=[
            "document",
            "part"
            ]
    )
    self.ensemble_retriever = EnsembleRetriever(
            retrievers=[
                self.bm25_retriever,
                self.db_retriever
                ],
            weights=self.weights,
            k=self.top_k
    )
    self.reranker = HuggingFaceCrossEncoder(
        model_name=reranker_name
    )

    self.llm = llm

  def ensemble_retrieve(
      self,
      query: str,
  ) -> List[Document]:

      return self.ensemble_retriever.invoke(query)

  def faiss_retrieve(
      self,
      query: str,
  ) -> List[Document]:

      return self.db_retriever.invoke(query)

  def bm25_retrieve(
          self,
          query: str,
  ) -> List[Document]:

      return self.bm25_retriever.invoke(query)

  def rerank_documents(
      self,
      query: str,
      docs: list[Document],
      top_k: int = 2
  ) -> list[Document]:

      pairs = [(query, doc.page_content) for doc in docs]
      scores = self.reranker.score(pairs)
      scored_docs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
      return [doc for doc, score in scored_docs[:top_k]]


  def calculate_hyde_embeddings(
      self,
      hyde_components
    ):
    input_embeddings = np.array(self.embedder.embed_query(" ".join((hyde_components['instruction_and_question']['instruction'], hyde_components['instruction_and_question']['question']))))
    generated_documents_embeddings = np.array(self.embedder.embed_documents(hyde_components['generated_documents']))
    hyde_embeddings = np.vstack([input_embeddings, generated_documents_embeddings]).mean(axis=0)
    return {"embeddings": list(hyde_embeddings),
            "instruction": hyde_components['instruction_and_question']['instruction'],
            "question": hyde_components['instruction_and_question']['question']
            }

  def get_relevant_documents(
      self,
      hyde_embeddings
    ):

    documents = self.vector_db.max_marginal_relevance_search_by_vector(
        hyde_embeddings['embeddings'],
        k = 10,
        fetch_k = 50
        )
    ensemble_docs = self.ensemble_retriever.invoke(" ".join([hyde_embeddings["question"], hyde_embeddings['question']]))

    for doc in ensemble_docs:
      if doc not in documents:
        documents.append(doc)


    return self.rerank_documents(" ".join([hyde_embeddings["question"], hyde_embeddings['question']]), documents)

  def format_docs(
      self,
      docs
  ):
    prompt = """Дальше приведены два релевантных для этого вопроса отрывка из российского законодательста:

### Первый релевантный отрывок из {}
{}
"{}"

### Второй релевантный отрывок из {}
{}
"{}"

Проанализируй всю предоставленную тебе информацию. Обязательно учитывай приведенные отрывки из законов и возвращайся к ним, при необходимости.
Твой ответ должен быть кратким и по существу, без лишних рассуждений.""".strip()

    doc_info = [element  for doc in docs for element in (doc.metadata['document'], doc.metadata['part'], doc.page_content)]
    context = prompt.format(*doc_info)
    return context

  def get_contexts(
      self,
      input_data: dict
  ) -> List[Document]:

    hyde_prompt_template = """
Ниже приведена инструкция, описывающая задачу, в сочетании с вводом, обеспечивающим дополнительный контекст. Напишите ответ, который соответствующим образом завершает запрос.

Ты - опытный юрист, специализирующийся на российском праве.
Твоя задача - сформулировать гипотетический фрагмент юридического текста (например, выдержку из закона, определение, статью), который мог бы содержать ответ на следующий юридический вопрос пользователя.
Этот гипотетический фрагмент будет использован для поиска похожих документов в базе данных.

Пожалуйста, представь свой гипотетический ответ в формальном юридическом стиле, как если бы это была часть официального документа.

### Контекст:
{instruction}

### Вопрос:
{question}

### Ответ:
    """.strip()
    instruction = input_data['instruction']
    question = input_data['question']
    hyde_prompt = ChatPromptTemplate.from_template(hyde_prompt_template)
    g_chain = (
        hyde_prompt
        | self.llm
        | StrOutputParser()
    )
    generated_answers = []
    for i in range(3):
      doc = g_chain.invoke({'instruction': instruction, 'question': question})
    # generated_documents = g_chain.batch([{'instruction': instruction, 'question': question}] * 3)
    # # print(generated_documents)
      response_start = doc.find("### Ответ:") + len("### Ответ:")

      generated_answers.append(doc[response_start:].strip())

    return generated_answers

  def get_answer(
      self,
      input_data: dict
  )-> str:

    rag_prompt_template = """
Ниже приведена инструкция, описывающая задачу, в сочетании с вводом, обеспечивающим дополнительный контекст. Напишите ответ, который соответствующим образом завершает запрос.

### Инструкция:
Ты - юридический консультант. Тебе даётся следующий контекст:
{instruction}

### Ввод:
{question}

### Для ответа на поставленный вопрос опирайся на предложенный ниже контекст:
{context}

### Ответ:
    """.strip()
    instruction = input_data['instruction']
    quesion = input_data['question']

    rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)
    retrieval_chain =  (
      RunnableParallel({
              'instruction_and_question': RunnablePassthrough(),
              'generated_documents': (RunnablePassthrough() | self.get_contexts)
          })
          | self.calculate_hyde_embeddings
          | self.get_relevant_documents
    )

    rag_chain = (
        {
            "context": retrieval_chain | self.format_docs,
            "instruction": lambda query: query['instruction'],
            "question": lambda query: query['question']
        }
        | rag_prompt
        | self.llm
        | StrOutputParser()
    )

    response = rag_chain.invoke(input_data)

    return response

### Импорт валидационного датасета

In [7]:
test_df = pd.read_csv("clear_test", index_col=0)
test_df.head()

,instruction,input,output
2588,Водитель ФИО1 был привлечен к административной...,Обязан ли инспектор проводить фотофиксацию мес...,"Нет, фотофиксация не является обязательной по ..."
6365,Инспектор ГИБДД составил протокол об администр...,Могут ли отменить постановление инспектора ГИБ...,"Да, могут. Согласно ст. 29.10 КоАП РФ, постано..."
10807,Сосед (ФИО1) конфликтовал с владелицей квартир...,"Можно ли избежать штрафа, если оскорбление был...","Нет, провокация не исключает ответственность, ..."
1120,"В результате ДТП, произошедшего из-за действий...","На какую неустойку я могу рассчитывать, если с...","По п. 21 ст. 12 ФЗ «Об ОСАГО», за каждый день ..."
204,АО «АльфаСтрахование» выплатило страховое возм...,Может ли страховая компания взыскать выплаченн...,"Да, может. Согласно п. 1 ст. 14 Федерального з..."


## Создание агента

In [8]:
legal_agent = LegalAgent(
    llm = llm,
    db_path = "./vector_db",
    embedder_name = "intfloat/multilingual-e5-large",
    reranker_name = "amberoad/bert-multilingual-passage-reranking-msmarco",
    top_k = 10
)

<ipython-input-6-d39ba09c139b>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embedder = HuggingFaceEmbeddings(


## Прогон на валидационном датасете

In [10]:
rag_answers = []
for instruction, question in tqdm(zip(test_df['instruction'],test_df['input'])):
  try:
    rag_answers.append(legal_agent.get_answer({"instruction":instruction, "question":question}))
  except:
    rag_answers.append('')
test_df['rag_answers'] = rag_answers

2it [00:52, 25.53s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
120it [55:45, 27.88s/it]


In [15]:
test_df.iloc[0]['rag_answers']

'Human: Ниже приведена инструкция, описывающая задачу, в сочетании с вводом, обеспечивающим дополнительный контекст. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\nТы - юридический консультант. Тебе даётся следующий контекст:\nВодитель ФИО1 был привлечен к административной ответственности по ч. 1 ст. 12.15 КоАП РФ за нарушение п. 9.10 ПДД РФ (несоблюдение безопасной дистанции), что привело к столкновению с автомобилем ФИО2. Инспектор ДПС вынес постановление о штрафе в 1500 рублей, которое было оставлено в силе решением районного суда. ФИО1 обжаловал эти акты, указывая на нарушения при составлении схемы ДТП, отсутствие фотофиксации, а также на вину второго участника аварии. Исходя из этого, ответь на вопрос:\n\n### Ввод:\nОбязан ли инспектор проводить фотофиксацию места ДТП?\n\n### Для ответа на поставленный вопрос опирайся на предложенный ниже контекст:\nДальше приведены два релевантных для этого вопроса отрывка из российского законодательста:\n\n

In [6]:
def clear_output(inp_str):
  response_start = inp_str.find("### Ответ:") + len("### Ответ:")
  return inp_str[response_start:].strip()

In [7]:
test_df['clear_answers'] = test_df['rag_answers'].apply(lambda x: clear_output(x))
test_df.head()

,instruction,input,output,rag_answers,clear_answers
2588,Водитель ФИО1 был привлечен к административной...,Обязан ли инспектор проводить фотофиксацию мес...,"Нет, фотофиксация не является обязательной по ...","Human: Ниже приведена инструкция, описывающая ...","Нет, обязанность проводиться фотофиксация не у..."
6365,Инспектор ГИБДД составил протокол об администр...,Могут ли отменить постановление инспектора ГИБ...,"Да, могут. Согласно ст. 29.10 КоАП РФ, постано...","Human: Ниже приведена инструкция, описывающая ...","Нет, постановление инспектора не будет отменен..."
10807,Сосед (ФИО1) конфликтовал с владелицей квартир...,"Можно ли избежать штрафа, если оскорбление был...","Нет, провокация не исключает ответственность, ...","Human: Ниже приведена инструкция, описывающая ...","Независимо от провокации потерпевшего, факт ос..."
1120,"В результате ДТП, произошедшего из-за действий...","На какую неустойку я могу рассчитывать, если с...","По п. 21 ст. 12 ФЗ «Об ОСАГО», за каждый день ...","Human: Ниже приведена инструкция, описывающая ...",Неустойка начисляется за **каждый день** проср...
204,АО «АльфаСтрахование» выплатило страховое возм...,Может ли страховая компания взыскать выплаченн...,"Да, может. Согласно п. 1 ст. 14 Федерального з...","Human: Ниже приведена инструкция, описывающая ...",Страховщик вправе требовать возмещения выплаче...


## Подсчёт BertScore относительно эталонных ответов

In [9]:
from bert_score import BERTScorer

scorer = BERTScorer(
    model_type="ai-forever/ru-en-RoSBERTa",
    num_layers=20,
    lang="ru"
)

def bertscorer(str1, str2):
  return float(scorer.score([str1], [str2])[2])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.49M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.99M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from tqdm import tqdm
import heapq
import numpy as np

retrieve_estimate = []
for n in tqdm(range(len(test_df))):

  F1 = bertscorer(test_df.iloc[n]['output'], test_df.iloc[n]['clear_answers'])
  retrieve_estimate.append(F1)

print(f"BertScore с эталонноым ответом: {np.mean(retrieve_estimate)}")

100%|██████████| 120/120 [00:04<00:00, 25.41it/s]

BertScore с эталонноым ответом: 0.7763332585493724
